In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nepsa/NepSA.csv
/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec.txt
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors_vocab.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_ngrams_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.syn1neg.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_vocab_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors_ngrams.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text


In [2]:
from pathlib import Path

df= pd.read_csv("/kaggle/input/nepsa/NepSA.csv")
df_copy= df.copy()
df.info()

scratch_output = "./scratch/"
finetuned_output = "./output"
Path(scratch_output).mkdir()
Path(finetuned_output).mkdir()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4707 non-null   object
 1   Target  4707 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.7+ KB


In [3]:
# Tokenizing using nltk and creating a corpus for models to train on

import nltk
from nltk.tokenize import word_tokenize

sentences= df_copy['Text'].apply(lambda x: word_tokenize(x)).tolist()

# Word2Vec scratch

In [4]:
from gensim.models import Word2Vec

scratch = Word2Vec(sentences, vector_size= 300,sg=1 ,negative=10 ,window= 15, min_count=3,alpha=0.0025, epochs=200, workers=4, seed= 42, compute_loss=True)  #sg=1 means using Skip-gram
print(scratch.get_latest_training_loss())

24787000.0


In [5]:
print(f"दलाल: {scratch.wv.most_similar('दलाल')}")
print(f"मुजि: {scratch.wv.most_similar('मुजि')}")
print(f"रन्डि: {scratch.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {scratch.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {scratch.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {scratch.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {scratch.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {scratch.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('गोरू', 0.6033902764320374), ('घुसिया', 0.5901913046836853), ('पापी', 0.5852494239807129), ('भारतिय', 0.5696837306022644), ('भुमाफिया', 0.5645279884338379), ('संसद', 0.5618158578872681), ('नपाएर', 0.5473347902297974), ('रहने', 0.547138512134552), ('सम्बन्ध', 0.5425519347190857), ('लाइक', 0.5418401956558228)]
मुजि: [('नगेन्द्र', 0.711967945098877), ('हरामि', 0.6714503169059753), ('रण्डि', 0.6606051921844482), ('चम्चा', 0.6590189337730408), ('खालि', 0.658369243144989), ('बादी', 0.6579492688179016), ('प्रचन्डे', 0.6545289754867554), ('आय', 0.6520548462867737), ('लामो', 0.6494603157043457), ('बोल्नै', 0.6484158039093018)]
रन्डि: [('रन्दि', 0.8004626631736755), ('राडी', 0.7874512076377869), ('मास्टर', 0.7845054864883423), ('बन', 0.7817702889442444), ('तेइ', 0.776989758014679), ('चिक्ने', 0.7758449912071228), ('जादा', 0.772700309753418), ('पढ्ने', 0.7706379294395447), ('बान', 0.7697343230247498), ('चिरि', 0.7646850943565369)]
चिक्ने: [('मया', 0.9028002023696899), ('मास्टर', 0.8564922

Saving Word2Vec will be done in txt as Word2Vec doesn't make sub word vectors unlike fasttext. So vectors saved as txt file will suffice.Furthermore the pre-trained Word2Vec model itself is a txt file and not a full model

In [6]:
scratch.wv.save_word2vec_format(os.path.join(scratch_output, 'scratch.txt'), binary=False)

# Pre-trained Word2Vec

In [7]:
from gensim.models import KeyedVectors

pretrained_input = "/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec.txt"

pretrained = KeyedVectors.load_word2vec_format(pretrained_input, binary=False)

# Fine-Tuned Word2Vec
The pretrained model is a txt file and finetuning without a full model doesn't sound convincing. However a full model from [NPVec1](https://github.com/nowalab/nepali-word-embeddings), specifically the tokenized and stemmed one which fits required case the most doesn't load at all. That's why the alternative being the 300 dimensional Word2Vec [model](https://ieee-dataport.org/open-access/300-dimensional-word-embeddings-nepali-language) which is only a text file of the word vectors. The finetuning procedure will be as stated in this [site](https://czarrar.github.io/Gensim-Word2Vec/) and another [site](https://datascience.stackexchange.com/questions/97568/fine-tuning-pre-trained-word2vec-model-with-gensim-4-0).

In [8]:
finetuned = Word2Vec(vector_size = 300,sg=1 ,negative=10 ,window= 15, min_count=3,alpha=0.0025, epochs=200, workers=4, seed= 42)
finetuned.build_vocab(sentences)

# Adding pre-trained model vocabulary
finetuned.build_vocab([list(pretrained.key_to_index.keys())], update=True)

# Ensuring all the words in the vocabulary are updated during finetuning with vectors of ones
vocab_size = len(finetuned.wv)
finetuned.wv.vectors_lockf = np.ones(vocab_size, dtype=np.float32)

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary then it is left as is in the original model
finetuned.wv.intersect_word2vec_format(pretrained_input, binary=False, lockf=1.0)
total_examples = finetuned.corpus_count

# Finally Finetuning
finetuned.train(sentences, total_examples=total_examples, epochs= finetuned.epochs)

(6063188, 9137400)

In [9]:
print(f"दलाल: {finetuned.wv.most_similar('दलाल')}")
print(f"मुजि: {finetuned.wv.most_similar('मुजि')}")
print(f"रन्डि: {finetuned.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {finetuned.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {finetuned.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {finetuned.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {finetuned.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {finetuned.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलाली', 0.5879755616188049), ('ठग', 0.5297701954841614), ('माफिया', 0.43256866931915283), ('भुमाफिया', 0.42711764574050903), ('फटाह', 0.41509851813316345), ('ठग्ने', 0.40759074687957764), ('लुटेर', 0.39497095346450806), ('धन्दा', 0.389771968126297), ('घुसखोरी', 0.38749760389328003), ('लुट्ने', 0.3824755847454071)]
मुजि: [('माचिक्ने', 0.9070309400558472), ('नालाएक', 0.8940120339393616), ('मुँजी', 0.89377361536026), ('लाडो', 0.8901323676109314), ('dig', 0.8819910883903503), ('नभुक', 0.8801674842834473), ('रन्डि', 0.8777561783790588), ('पुती', 0.8700353503227234), ('चिक्ने', 0.8696476221084595), ('10', 0.8682755827903748)]
रन्डि: [('चिक्ने', 0.9650057554244995), ('रन्दि', 0.9624672532081604), ('मुँजी', 0.9508404731750488), ('मादरचोद', 0.9497822523117065), ('जिउदै', 0.949225127696991), ('माचिक्ने', 0.948803186416626), ('राडि', 0.9486873149871826), ('सिदै', 0.9470410346984863), ('बिस्वराज', 0.9460260272026062), ('मजी', 0.9448207020759583)]
चिक्ने: [('जिउदै', 0.9845611453056335), ('

In [10]:
finetuned.wv.save_word2vec_format(os.path.join(finetuned_output, 'finetuned.txt'), binary=False)

In [11]:
# keeping this at last as this will raise Exception and stop the script. So that all the models are saved before the exception
print(f"दलाल: {pretrained.most_similar('दलाल')}")
print(f"मुजि: {pretrained.most_similar('मुजि')}")
print(f"रन्डि: {pretrained.most_similar('रन्डि')}")
print(f"चिक्ने: {pretrained.most_similar('चिक्ने')}")
print(f"ठोक्ने: {pretrained.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {pretrained.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {pretrained.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {pretrained.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलालहरू', 0.6451959013938904), ('दलाली', 0.5998064875602722), ('बिचौलिया', 0.5902517437934875), ('पुँजीपति', 0.5644993782043457), ('ठग', 0.5342469215393066), ('एजेन्ट', 0.5191587805747986), ('पूँजीपति', 0.5182876586914062), ('नोकरशाही', 0.5007683634757996), ('कारोबारी', 0.4855310916900635), ('गिरोह', 0.4851357042789459)]


KeyError: "Key 'मुजि' not present in vocabulary"

As observed the model doesn't have word vectors for profane word, which is expected as the model was trained on new sources and wikipedia.